In [163]:
import numpy as np
import pandas as pd
import os
os.getcwd()

'/Users/Alvaro/Library/Mobile Documents/com~apple~CloudDocs/DTU/Autumn 2023/Phage project/phage/notebooks'

In [189]:
df = pd.read_csv("../data/raw/inphared_8Sep2023/8Sep2023_data.tsv", sep="\t")
df.rename(columns={'Isolation Host (beware inconsistent and nonsense values)': 'Isolation Host'}, inplace=True)
print("Shape before filtering", df.shape)
df = df[['Accession','Host','Isolation Host']]
df = df[df['Host'] != "Unspecified"] 
df['Host'] = df['Host'].replace('Enteroccous', 'Enterococcus')
print("Shape after filtering unspecified hosts", df.shape)
df = df[df['Isolation Host'] != "Unspecified"]
print("Shape after filtering unspecified isolation hosts", df.shape)
df.head()

Shape before filtering (26287, 27)
Shape after filtering unspecified hosts (23367, 3)
Shape after filtering unspecified isolation hosts (14341, 3)


,Accession,Host,Isolation Host
31,MN335248,Vibrio,Xanthomonas vesicatoria
32,MK250029,Prevotella,Prevotella sp.
33,MK250028,Prevotella,Prevotella sp.
34,MK250027,Prevotella,Prevotella sp.
35,MK250026,Prevotella,Prevotella sp.


In [165]:
all_gram = pd.read_csv("../data/interim/all_gram_stains_fixed_species.csv")
all_gram.head()

,species,Gram stain,Genus
0,Vallitalea guaymasensis,positive,Vallitalea
1,Acaricomes phytoseiuli,positive,Acaricomes
2,Acetanaerobacterium elongatum,positive,Acetanaerobacterium
3,Acetilactobacillus jinshanensis,positive,Acetilactobacillus
4,Acetivibrio mesophilus,positive,Acetivibrio


# Excluding duplicated genera and species

In [190]:
exclude_genus = ['Clostridium', 'Neobacillus', 'Alteribacter', 'Desulfotomaculum', 'Caloramator', 'Desulforamulus', 'Heyndrickxia', 'Peptoclostridium', 'Thermoanaerobacter', 'Thermoanaerobacterium', 'Aureimonas', 'Actinomadura', 'Alkalibacterium', 'Deinococcus', 'Tepidibacillus', 'Sphingomonas', 'Lysinibacillus', 'Ruminiclostridium', 'Caldicellulosiruptor', 'Pseudomonas', 'Streptococcus', 'Microlunatus', 'Streptomyces', 'Butyricimonas', 'Halalkalibacter', 'Chelativorans', 'Natrinema', 'Ureibacillus', 'Clostridioides', 'Desulfosporosinus', 'Lacibacter', 'Nocardioides', 'Siminovitchia', 'Belliella', 'Tistlia', 'Actinoplanes', 'Paenibacillus', 'Vallitalea', 'Actinotalea', 'Cohnella', 'Rhizobium', 'Anaerotignum', 'Cellulomonas', 'Flavobacterium', 'Bacillus', 'Nesterenkonia']
exclude_species = ['Ureibacillus massiliensis', 'Tistlia consotensis', 'Clostridioides difficile', 'Vallitalea guaymasensis', 'Belliella pelovolcani', 'Actinotalea ferrariae']

print("Shape of the dataframe before deleting the common genera: ", df.shape)
df = df[~df['Host'].isin(exclude_genus)]
print("Shape of the dataframe after deleting the common genera: ", df.shape)
df = df[~df['Isolation Host'].isin(exclude_species)]
shape1 = df.shape
print("Shape of the dataframe after deleting the common species: ", shape1[1])


Shape of the dataframe before deleting the common genera:  (14341, 3)
Shape of the dataframe after deleting the common genera:  (11373, 3)
Shape of the dataframe after deleting the common species:  3


# First mapping: species to isolation host

In [191]:
# Only keep first two strings in each row of Isolation Host to reduce name incompatibilities
# Split each value on space and select the first two elements
df['Isolation Host'] = df['Isolation Host'].str.split().str[:2].str.join(' ')
df

,Accession,Host,Isolation Host
31,MN335248,Vibrio,Xanthomonas vesicatoria
32,MK250029,Prevotella,Prevotella sp.
33,MK250028,Prevotella,Prevotella sp.
34,MK250027,Prevotella,Prevotella sp.
35,MK250026,Prevotella,Prevotella sp.
...,...,...,...
26263,AJ854042,Acidianus,Acidianus pozzuoliensis
26266,DQ490056,Lactococcus,Lactococcus lactis
26274,AJ414696,Sulfolobus,Sulfolobus islandicus
26275,AJ344259,Sulfolobus,Sulfolobus islandicus


In [192]:
# Create a mapping dictionary from species to Gram stain
species_to_stain = dict(zip(all_gram['species'], all_gram['Gram stain']))
species_to_stain

{'Vallitalea guaymasensis': 'positive',
 'Acaricomes phytoseiuli': 'positive',
 'Acetanaerobacterium elongatum': 'positive',
 'Acetilactobacillus jinshanensis': 'positive',
 'Acetivibrio mesophilus': 'positive',
 'Acetivibrio saccincola': 'positive',
 'Acidimicrobium ferrooxidans': 'positive',
 'Acidipropionibacterium olivae': 'positive',
 'Acidipropionibacterium virtanenii': 'positive',
 'Acrocarpospora phusangensis': 'positive',
 'Actinoallomurus acaciae': 'positive',
 'Actinoallomurus acanthiterrae': 'positive',
 'Actinoallomurus amamiensis': 'positive',
 'Actinoallomurus caesius': 'positive',
 'Actinoallomurus fulvus': 'positive',
 'Actinoallomurus iriomotensis': 'positive',
 'Actinoallomurus luridus': 'positive',
 'Actinoallomurus oryzae': 'positive',
 'Actinoallomurus purpureus': 'positive',
 'Actinoallomurus spadix': 'positive',
 'Actinoallomurus yoronensis': 'positive',
 'Actinoalloteichus spitiensis': 'positive',
 'Actinobaculum massiliense': 'positive',
 'Actinobaculum suis':

In [193]:
# Use the mapping to create the 'staining' column in df
df_mapped = df.copy(deep=True)
df_mapped['staining'] = df_mapped['Isolation Host'].map(species_to_stain)
drop = df_mapped.dropna(subset=['staining'])
shape2 = drop.shape
print(shape2)
print("Percentage of phages assigned a class based solely on species: ", str(shape2[0]/shape1[0]*100))
print("Percentage of the all_gram dataset used to assign a class based solely on species: ", str(shape2[0]/all_gram.shape[0]*100))

(2558, 4)
Percentage of phages assigned a class based solely on species:  22.491866701837683
Percentage of the all_gram dataset used to assign a class based solely on species:  33.92123060602042


# Second mapping: using genus and Host

In [194]:
# Create a mapping dictionary from species to Gram stain
genus_to_stain = dict(zip(all_gram['Genus'], all_gram['Gram stain']))
genus_to_stain

{'Vallitalea': 'negative',
 'Acaricomes': 'positive',
 'Acetanaerobacterium': 'positive',
 'Acetilactobacillus': 'positive',
 'Acetivibrio': 'positive',
 'Acidimicrobium': 'positive',
 'Acidipropionibacterium': 'positive',
 'Acrocarpospora': 'positive',
 'Actinoallomurus': 'positive',
 'Actinoalloteichus': 'positive',
 'Actinobaculum': 'positive',
 'Actinocatenispora': 'positive',
 'Actinocorallia': 'positive',
 'Actinocrinis': 'positive',
 'Actinocrispum': 'positive',
 'Actinokineospora': 'positive',
 'Actinomadura': 'negative',
 'Actinomyces': 'positive',
 'Actinomycetospora': 'positive',
 'Actinophytocola': 'positive',
 'Actinoplanes': 'negative',
 'Actinopolymorpha': 'positive',
 'Actinopolyspora': 'positive',
 'Actinorectispora': 'positive',
 'Actinorhabdospora': 'positive',
 'Actinospica': 'positive',
 'Actinosynnema': 'positive',
 'Actinotalea': 'positive',
 'Actinotignum': 'positive',
 'Acutalibacter': 'positive',
 'Adlercreutzia': 'positive',
 'Aeribacillus': 'positive',
 'Aer

In [195]:
# Use the mapping to create the 'staining' column in df
df_mapped['staining'] = df_mapped['Host'].map(genus_to_stain)
drop = df_mapped.dropna(subset=['staining'])
shape3 = drop.shape
print(shape3)
print("Percentage of phages assigned a class using species+genus: ", str(shape3[0]/shape1[0]*100))
print("Percentage of the all_gram dataset used to assign a class based on species+genus: ", str(shape3[0]/all_gram.shape[0]*100))

(9050, 4)
Percentage of phages assigned a class using species+genus:  79.57443066912863
Percentage of the all_gram dataset used to assign a class based on species+genus:  120.01060867258984


# Exploring the remaining phages with no assignation

In [196]:
remaining = df_mapped[(df_mapped["staining"] != "positive") & (df_mapped["staining"] != "negative")]
remaining.head()

,Accession,Host,Isolation Host,staining
69,OR413347,Salmonella,Salmonella enterica,NaN
70,OR413345,Salmonella,Salmonella enterica,NaN
71,OR413344,Salmonella,Salmonella enterica,NaN
72,OR413342,Salmonella,Salmonella enterica,NaN
2301,OR067837,Salmonella,Salmonella enterica,NaN


In [197]:
print(len(remaining["Host"].unique().tolist()))
print(remaining["Host"].unique().tolist())

88
['Salmonella', 'Synechococcus', 'Pseudanabaena', 'Mammaliicoccus', 'Mycobacteroides', 'Nostoc', 'Microcystis', 'Methanosarcina', 'Actinomycetia', 'Saccharolobus', 'Cylindrospermopsis', 'Sulfolobus', 'Metallosphaera', 'Liberibacter', 'Enterobacteria', 'Megaira', 'Dipodfec', 'Thermoproteus', 'Pyrobaculum', 'Methanocaldococcus', 'Acidianus', 'Spiroplasma', 'Elizabethkingia', 'Myxococcus', 'Janthinobacterium', 'Pelagibacter', 'Brochothrix', 'Sinorhizobium', 'Butyrivibrio', 'Lentibacter', 'Salinibacter', 'Eggerthella', 'Phormidium', 'Mycoplasma', 'Chlamydia', 'Hamiltonella', 'Edwardsiella', 'Nodularia', 'Raoultella', 'Faecalibacterium', 'Leclercia', 'Azobacteroides', 'Curvibacter', 'Morganella', 'Natrialba', 'Verrucomicrobia', 'Prochlorococcus', 'Hydrogenobaculum', 'Anabaena', 'Croceibacter', 'Oenococcus', 'Puniceispirillum', 'Tetrasphaera', 'Bdellovibrio', 'Thermococcus', 'Planktothrix', 'Sodalis', 'Silicibacter', 'Pyrococcus', 'Listonella', 'Kluyvera', 'Hungatella', 'Amedibacillus', 'L

In [198]:
print(len(remaining["Isolation Host"].unique().tolist()))
print(remaining["Isolation Host"].unique().tolist())

158
['Salmonella enterica', 'Parasynechococcus marenigrum', 'Synechococcus sp.', 'Salmonella sp.', 'Pseudanabaena sp.', 'Mammaliicoccus sciuri', 'Mycobacteroides abscessus', 'Nostoc sp.', 'Microcystis sp.', 'Methanosarcina mazei', 'Actinomycetia bacterium', 'Saccharolobus shibatae', 'Cylindrospermopsis raciborskii', 'Sulfolobus beitou', 'Metallosphaera prunae', 'Candidatus Liberibacter', 'Escherichia coli', 'Candidatus Megaira', 'Dipodomys merriami', 'Thermoproteus sp.', 'Pyrobaculum arsenaticum', 'Methanocaldococcus fervens', 'Saccharolobus solfataricus', 'Metallosphaera sedula', 'Acidianus brierleyi', 'Circulifer haematoceps', 'Salmonella gallinarum', 'Sulfolobus tengchongensis', 'Elizabethkingia anophelis', 'Myxococcus xanthus', 'Janthinobacterium lividum', 'Sulfolobus sp.', 'Sulfolobus', 'Candidadtus Pelagibacter', 'Dipodomys merriami;', 'Brochothrix thermosphacta', 'Sinorhizobium meliloti', 'Butyrivibrio fibrisolvens', 'Lentibacter sp.', 'Synechococcus', 'Salinibacter ruber', 'Egg

# Obtaining the remaining classification with Thomas pickle dict

In [199]:
import pickle

with open('../data/external/gram_pos_neg_dict.pkl', 'rb') as file:
    phage_dict = pickle.load(file)

file.close()

# Create a new genus_to_stain with updated values
phage_dict = {key: 'negative' if value == 'neg' else 'positive' for key, value in phage_dict.items()}

for key, value in list(phage_dict.items())[:3]:
    print(f'{key}: {value}')


Actinobacteria: positive
Actinomycetales: positive
Bifidobacterium: positive


Update the dataframe with Thomas dict

In [200]:
# Iterate through the DataFrame
for index, row in df_mapped.iterrows():
    if pd.isna(row['staining']):  # Check if 'staining' column is NaN
        host = row['Host']
        if host in phage_dict:
            df_mapped.at[index, 'staining'] = phage_dict[host]

drop = df_mapped.dropna(subset=['staining'])
shape4 = drop.shape
print(shape4)
print("Percentage of phages assigned a classusing species+genus+thomas dict: ", str(shape4[0]/shape1[0]*100))

(10485, 4)
Percentage of phages assigned a classusing species+genus+thomas dict:  92.19203376417832


In [201]:
remaining = df_mapped[df_mapped['staining'].isna()]
print(len(remaining["Host"].unique().tolist()))
print(remaining["Host"].unique().tolist())

57
['Pseudanabaena', 'Mammaliicoccus', 'Mycobacteroides', 'Nostoc', 'Methanosarcina', 'Actinomycetia', 'Saccharolobus', 'Cylindrospermopsis', 'Sulfolobus', 'Metallosphaera', 'Liberibacter', 'Enterobacteria', 'Megaira', 'Dipodfec', 'Thermoproteus', 'Pyrobaculum', 'Methanocaldococcus', 'Acidianus', 'Spiroplasma', 'Myxococcus', 'Janthinobacterium', 'Butyrivibrio', 'Salinibacter', 'Mycoplasma', 'Chlamydia', 'Faecalibacterium', 'Leclercia', 'Azobacteroides', 'Prochlorococcus', 'Hydrogenobaculum', 'Anabaena', 'Puniceispirillum', 'Thermococcus', 'Silicibacter', 'Pyrococcus', 'Listonella', 'Kluyvera', 'Hungatella', 'Amedibacillus', 'Leptolyngbya', 'Hujiaoplasma', 'Poriferisphaera', 'Plectonema', 'Cedecea', 'Citromicrobium', 'Kurthia', 'Lokiarchaeota', 'Pararheinheimera', 'Lentisphaerae', 'Solobacterium', 'Arthronema', 'Mastigocladus', 'Acholeplasma', 'Aphanizomenon', 'Xenohaliotis', 'Pelagibaca', 'Nitrososphaera']


In [202]:
print(len(remaining["Isolation Host"].unique().tolist()))
print(remaining["Isolation Host"].unique().tolist())

95
['Pseudanabaena sp.', 'Mammaliicoccus sciuri', 'Mycobacteroides abscessus', 'Nostoc sp.', 'Methanosarcina mazei', 'Actinomycetia bacterium', 'Saccharolobus shibatae', 'Cylindrospermopsis raciborskii', 'Sulfolobus beitou', 'Metallosphaera prunae', 'Candidatus Liberibacter', 'Escherichia coli', 'Candidatus Megaira', 'Dipodomys merriami', 'Thermoproteus sp.', 'Pyrobaculum arsenaticum', 'Methanocaldococcus fervens', 'Saccharolobus solfataricus', 'Metallosphaera sedula', 'Acidianus brierleyi', 'Circulifer haematoceps', 'Sulfolobus tengchongensis', 'Myxococcus xanthus', 'Janthinobacterium lividum', 'Sulfolobus sp.', 'Sulfolobus', 'Dipodomys merriami;', 'Butyrivibrio fibrisolvens', 'Salinibacter ruber', 'Mycoplasmopsis pulmonis', 'Chlamydophila pneumoniae', 'Faecalibacterium prausnitzii', 'Leclercia adecarboxylata', 'Candidatus Azobacteroides', 'Metallosphaera sp.', 'Sulfolobus islandicus', 'Acidianus sp.', 'E. coli', 'Salmonella enterica', 'Sulfolobales', 'Prochlorococcus sp.', 'Hydrogeno

### Figuring out the remaining hosts

In [203]:
# List of genera with possible misspellings
genera_list = ['Pseudanabaena', 'Mammaliicoccus', 'Enteroccous', 'Mycobacteroides', 'Nostoc', 'Methanosarcina', 'Actinomycetia', 'Saccharolobus', 'Cylindrospermopsis', 'Sulfolobus', 'Metallosphaera', 'Liberibacter', 'Enterobacteria', 'Megaira', 'Dipodfec', 'Thermoproteus', 'Pyrobaculum', 'Methanocaldococcus', 'Acidianus', 'Spiroplasma', 'Myxococcus', 'Janthinobacterium', 'Butyrivibrio', 'Salinibacter', 'Mycoplasma', 'Chlamydia', 'Faecalibacterium', 'Leclercia', 'Azobacteroides', 'Prochlorococcus', 'Hydrogenobaculum', 'Anabaena', 'Puniceispirillum', 'Thermococcus', 'Silicibacter', 'Pyrococcus', 'Listonella', 'Kluyvera', 'Hungatella', 'Amedibacillus', 'Leptolyngbya', 'Hujiaoplasma', 'Poriferisphaera', 'Plectonema', 'Cedecea', 'Citromicrobium', 'Kurthia', 'Lokiarchaeota', 'Pararheinheimera', 'Lentisphaerae', 'Solobacterium', 'Arthronema', 'Mastigocladus', 'Acholeplasma', 'Aphanizomenon', 'Xenohaliotis', 'Pelagibaca', 'Nitrososphaera']

# Reference list of correctly spelled genera (you can create your own or use a reliable source)
reference_genera = ['Pseudanabaena', 'Mammaliicoccus', 'Enterococcus', 'Mycobacteroides', 'Nostoc', 'Methanosarcina', 'Actinomycetia', 'Sulfolobus', 'Metallosphaera', 'Liberibacter', 'Enterobacter', 'Megaira', 'Thermoproteus', 'Pyrobaculum', 'Methanocaldococcus', 'Acidianus', 'Spiroplasma', 'Myxococcus', 'Janthinobacterium', 'Butyrivibrio', 'Salinibacter', 'Mycoplasma', 'Chlamydia', 'Faecalibacterium', 'Leclercia', 'Azobacteroides', 'Prochlorococcus', 'Hydrogenobaculum', 'Anabaena', 'Puniceispirillum', 'Thermococcus', 'Silicibacter', 'Pyrococcus', 'Listonella', 'Kluyvera', 'Hungatella', 'Amedibacillus', 'Leptolyngbya', 'Hujiaoplasma', 'Poriferisphaera', 'Plectonema', 'Cedecea', 'Citromicrobium', 'Kurthia', 'Lokiarchaeota', 'Pararheinheimera', 'Lentisphaerae', 'Solobacterium', 'Arthronema', 'Mastigocladus', 'Acholeplasma', 'Aphanizomenon', 'Xenohaliotis', 'Pelagibaca', 'Nitrososphaera']

# Find misspelled genera
misspelled_genera = set(genera_list) - set(reference_genera)

# Print misspelled genera
print("Misspelled Genera:")
print(misspelled_genera)


Misspelled Genera:
{'Dipodfec', 'Saccharolobus', 'Enterobacteria', 'Enteroccous', 'Cylindrospermopsis'}


In [207]:
print(genera_list)

['Pseudanabaena', 'Mammaliicoccus', 'Enteroccous', 'Mycobacteroides', 'Nostoc', 'Methanosarcina', 'Actinomycetia', 'Saccharolobus', 'Cylindrospermopsis', 'Sulfolobus', 'Metallosphaera', 'Liberibacter', 'Enterobacteria', 'Megaira', 'Dipodfec', 'Thermoproteus', 'Pyrobaculum', 'Methanocaldococcus', 'Acidianus', 'Spiroplasma', 'Myxococcus', 'Janthinobacterium', 'Butyrivibrio', 'Salinibacter', 'Mycoplasma', 'Chlamydia', 'Faecalibacterium', 'Leclercia', 'Azobacteroides', 'Prochlorococcus', 'Hydrogenobaculum', 'Anabaena', 'Puniceispirillum', 'Thermococcus', 'Silicibacter', 'Pyrococcus', 'Listonella', 'Kluyvera', 'Hungatella', 'Amedibacillus', 'Leptolyngbya', 'Hujiaoplasma', 'Poriferisphaera', 'Plectonema', 'Cedecea', 'Citromicrobium', 'Kurthia', 'Lokiarchaeota', 'Pararheinheimera', 'Lentisphaerae', 'Solobacterium', 'Arthronema', 'Mastigocladus', 'Acholeplasma', 'Aphanizomenon', 'Xenohaliotis', 'Pelagibaca', 'Nitrososphaera']


In [185]:
remaining_genera_dict = {'Pseudanabaena': 'negative',
 'Mammaliicoccus': 'positive',
 'Mycobacteroides': 'positive',
 'Nostoc': 'negative',
 'Methanosarcina': 'positive',
 'Actinomycetia': 'negative',
 'Saccharolobus': 'negative',
 'Cylindrospermopsis': 'positive',
 'Sulfolobus': 'negative',
 'Metallosphaera': 'positive',
 'Liberibacter': 'negative',
 'Enterobacteria': 'negative',
 'Megaira': 'positive',
 'Dipodfec': 'positive',
 'Thermoproteus': 'negative',
 'Pyrobaculum': 'negative',
 'Methanocaldococcus': 'positive',
 'Acidianus': 'positive',
 'Spiroplasma': 'negative',
 'Myxococcus': 'negative',
 'Janthinobacterium': 'negative',
 'Butyrivibrio': 'negative',
 'Salinibacter': 'positive',
 'Mycoplasma': 'positive',
 'Chlamydia': 'negative',
 'Faecalibacterium': 'positive',
 'Leclercia': 'negative',
 'Azobacteroides': 'positive',
 'Prochlorococcus': 'negative',
 'Hydrogenobaculum': 'negative',
 'Anabaena': 'negative',
 'Puniceispirillum': 'negative',
 'Thermococcus': 'negative',
 'Silicibacter': 'negative',
 'Pyrococcus': 'negative',
 'Listonella': 'positive',
 'Kluyvera': 'negative',
 'Hungatella': 'negative',
 'Amedibacillus': 'positive',
 'Leptolyngbya': 'negative',
 'Hujiaoplasma': 'negative',
 'Poriferisphaera': 'positive',
 'Plectonema': 'positive',
 'Cedecea': 'negative',
 'Citromicrobium': 'positive',
 'Kurthia': 'negative',
 'Lokiarchaeota': 'negative',
 'Pararheinheimera': 'positive',
 'Lentisphaerae': 'negative',
 'Solobacterium': 'positive',
 'Arthronema': 'negative',
 'Mastigocladus': 'positive',
 'Acholeplasma': 'negative',
 'Aphanizomenon': 'positive',
 'Xenohaliotis': 'negative',
 'Pelagibaca': 'positive',
 'Nitrososphaera': 'negative'}

{'Pseudanabaena': 'negative',
 'Mammaliicoccus': 'positive',
 'Enteroccous': 'negative',
 'Mycobacteroides': 'negative',
 'Nostoc': 'positive',
 'Methanosarcina': 'positive',
 'Actinomycetia': 'negative',
 'Saccharolobus': 'negative',
 'Cylindrospermopsis': 'positive',
 'Sulfolobus': 'negative',
 'Metallosphaera': 'positive',
 'Liberibacter': 'negative',
 'Enterobacteria': 'negative',
 'Megaira': 'positive',
 'Dipodfec': 'positive',
 'Thermoproteus': 'negative',
 'Pyrobaculum': 'negative',
 'Methanocaldococcus': 'positive',
 'Acidianus': 'positive',
 'Spiroplasma': 'negative',
 'Myxococcus': 'negative',
 'Janthinobacterium': 'negative',
 'Butyrivibrio': 'negative',
 'Salinibacter': 'positive',
 'Mycoplasma': 'positive',
 'Chlamydia': 'negative',
 'Faecalibacterium': 'positive',
 'Leclercia': 'negative',
 'Azobacteroides': 'positive',
 'Prochlorococcus': 'negative',
 'Hydrogenobaculum': 'negative',
 'Anabaena': 'negative',
 'Puniceispirillum': 'negative',
 'Thermococcus': 'negative',
 '

In [205]:
df[df['Host'] == 'Methanosarcina']

,Accession,Host,Isolation Host
2567,OR220913,Methanosarcina,Methanosarcina mazei
2568,OR220912,Methanosarcina,Methanosarcina mazei
2569,OR220911,Methanosarcina,Methanosarcina mazei
18079,MK170447,Methanosarcina,Methanosarcina sp.
22427,MF186604,Methanosarcina,Methanosarcina sp.


# Updating Thomas Dictionary

In [140]:
import pickle

with open('../data/external/gram_pos_neg_dict.pkl', 'rb') as file:
    phage_dict = pickle.load(file)

file.close()

Seeing which elements one list of genera has that the other does not and vice versa

In [141]:
phage_df = pd.DataFrame(phage_dict.items(), columns=['Genus', 'Label'])
phage_list = phage_df['Genus'].to_list()
phage_list

['Actinobacteria',
 'Actinomycetales',
 'Bifidobacterium',
 'Brevibacterium',
 'Cellulomonas',
 'Clavibacter',
 'Corynebacterium',
 'Dermatophilus',
 'Frankia',
 'Gardnerella',
 'Gordonia',
 'Microbacterium',
 'Micrococcaceae',
 'Micromonosporaceae',
 'Mycobacteriaceae',
 'Nocardiaceae',
 'Nocardioides',
 'Nocardiopsis',
 'Propionibacteriaceae',
 'Pseudonocardia',
 'Saccharopolyspora',
 'Streptomycetaceae',
 'Tropheryma',
 'Bacillales',
 'Alicyclobacillus',
 'Bacillaceae',
 'Brevibacillus',
 'Brochothrix',
 'Gemella',
 'Listeria',
 'Paenibacillus',
 'Pasteuria',
 'Staphylococcaceae',
 'Thermoactinomyces',
 'Gram-Positive',
 'Aerococcus',
 'Deinococcus',
 'Oenococcus',
 'Peptococcaceae',
 'Peptostreptococcus',
 'Planococcaceae',
 'Ruminococcus',
 'Sarcina',
 'Streptococcaceae',
 'Sporosarcina',
 'Lactobacillales',
 'Aerococcaceae',
 'Carnobacteriaceae',
 'Enterococcaceae',
 'Lactobacillaceae',
 'Leuconostocaceae',
 'Aquifex',
 'Arcobacter',
 'Bacteroidetes',
 'Bacteroidaceae',
 'Cytopha

In [142]:
all_gram_list = list(all_gram['Genus'].unique())
all_gram_list

['Vallitalea',
 'Acaricomes',
 'Acetanaerobacterium',
 'Acetilactobacillus',
 'Acetivibrio',
 'Acidimicrobium',
 'Acidipropionibacterium',
 'Acrocarpospora',
 'Actinoallomurus',
 'Actinoalloteichus',
 'Actinobaculum',
 'Actinocatenispora',
 'Actinocorallia',
 'Actinocrinis',
 'Actinocrispum',
 'Actinokineospora',
 'Actinomadura',
 'Actinomyces',
 'Actinomycetospora',
 'Actinophytocola',
 'Actinoplanes',
 'Actinopolymorpha',
 'Actinopolyspora',
 'Actinorectispora',
 'Actinorhabdospora',
 'Actinospica',
 'Actinosynnema',
 'Actinotalea',
 'Actinotignum',
 'Acutalibacter',
 'Adlercreutzia',
 'Aeribacillus',
 'Aeriscardovia',
 'Aerococcus',
 'Aeromicrobium',
 'Aestuariimicrobium',
 'Agathobaculum',
 'Agrilactobacillus',
 'Agrococcus',
 'Agromyces',
 'Aldersonia',
 'Alicyclobacillus',
 'Alkalibacillus',
 'Alkalibacterium',
 'Alkalicoccobacillus',
 'Alkalicoccus',
 'Alkalihalobacillus',
 'Alkalihalophilus',
 'Alkaliphilus',
 'Alkalitalea',
 'Allobacillus',
 'Allobranchiibius',
 'Allocatelligl

In [143]:
unique_to_list1 = list(set(all_gram_list) - set(phage_list))
unique_to_list2 = list(set(phage_list) - set(all_gram_list))

print("Elements unique to all_gram_list:", unique_to_list1)
print("Elements unique to phage_list:", unique_to_list2)

Elements unique to all_gram_list: ['Hathewaya', 'Luteivirga', 'Wocania', 'Pseudobizionia', 'Alloscardovia', 'Kingella', 'Abyssibacter', 'Chthonobacter', 'Pseudoruegeria', 'Anaerosacchariphilus', 'Beijerinckia', 'Thiothrix', 'Falsochrobactrum', 'Crenobacter', 'Dehalogenimonas', 'Aestuariibaculum', 'Roseiflexus', 'Litorilinea', 'Methylonatrum', 'Actinokineospora', 'Methanoregula', 'Salinisphaera', 'Oceanidesulfovibrio', 'Kroppenstedtia', 'Caldanaerobius', 'Anoxybacillus', 'Crocinitomix', 'Spongiibacter', 'Dietzia', 'Ferruginibacter', 'Promicromonospora', 'Rhodoligotrophos', 'Namhaeicola', 'Pseudorhodobacter', 'Tumebacillus', 'Propioniciclava', 'Niveibacterium', 'Agaricicola', 'Izhakiella', 'Companilactobacillus', 'Methylophilus', 'Atopococcus', 'Pseudogemmobacter', 'Dactylosporangium', 'Anaerohalosphaera', 'Actinocrinis', 'Collinsella', 'Gracilibacillus', 'Bacterioplanoides', 'Sulfobacillus', 'Lichenibacterium', 'Spiribacter', 'Desulfatiferula', 'Actinacidiphila', 'Lysobacter', 'Stappia'

In [144]:
if 'Salmonella' in phage_list:
    print("Salmonella is in the list.")
else:
    print("Salmonella is not in the list.")

Salmonella is in the list.


In [145]:
# Create a new genus_to_stain with updated values
genus_to_stain2 = {key: 'neg' if value == 'negative' else 'pos' for key, value in genus_to_stain.items()}

Comparing both dictionaries to know what is missing in each

In [146]:
# Find common keys
common_keys = set(genus_to_stain2.keys()) & set(phage_dict.keys())

# Initialize a list to store the keys that have different values
different_values_keys = []

# Iterate through common keys and check if the values are equal
for key in common_keys:
    if genus_to_stain2[key] != phage_dict[key]:
        different_values_keys.append(key)

print(different_values_keys)
print()
# # Print the common keys and their values in both dictionaries
# for key in common_keys:
#     print(f'Key: {key}, Dictionary 1 Value: {dict1[key]}, Dictionary 2 Value: {dict2[key]}')

# Print keys with different values
print("\nKeys with different values:")
# for key in different_values_keys:
#     print(key)

# Iterate through the keys in dict1
for key in genus_to_stain2:
    # Check if the key is in dict2 and if the values are different
    if key in phage_dict and genus_to_stain2[key] != phage_dict[key]:
        print(f'Key: {key}, Bacdive Value: {genus_to_stain2[key]}, Thomas Value: {phage_dict[key]}','\n')


['Phaeobacter', 'Mesorhizobium', 'Peptoclostridium', 'Caldibacillus', 'Paenibacillus', 'Nocardioides', 'Exiguobacterium', 'Deinococcus', 'Thermoanaerobacterium', 'Lysinibacillus', 'Cellulomonas', 'Gardnerella', 'Methanobacterium']


Keys with different values:
Key: Cellulomonas, Bacdive Value: neg, Thomas Value: pos 

Key: Deinococcus, Bacdive Value: neg, Thomas Value: pos 

Key: Exiguobacterium, Bacdive Value: pos, Thomas Value: neg 

Key: Lysinibacillus, Bacdive Value: neg, Thomas Value: pos 

Key: Nocardioides, Bacdive Value: neg, Thomas Value: pos 

Key: Paenibacillus, Bacdive Value: neg, Thomas Value: pos 

Key: Peptoclostridium, Bacdive Value: neg, Thomas Value: pos 

Key: Thermoanaerobacterium, Bacdive Value: neg, Thomas Value: pos 

Key: Caldibacillus, Bacdive Value: neg, Thomas Value: pos 

Key: Gardnerella, Bacdive Value: neg, Thomas Value: pos 

Key: Mesorhizobium, Bacdive Value: neg, Thomas Value: pos 

Key: Methanobacterium, Bacdive Value: neg, Thomas Value: pos 

Key: Pha

# Updating Thomas dictionary

In [132]:

missing_keys = set(genus_to_stain2.keys()) - set(phage_dict.keys())

# Get the shape of dict2 before the addition
print("Length of dictionary before adding mine: ",len(phage_dict))
phage_dict_updated = phage_dict

# Add missing key-value pairs to dict2
for key in missing_keys:
    phage_dict_updated[key] = genus_to_stain2[key]

phage_dict_updated = dict(sorted(phage_dict_updated.items()))

# Get the shape of dict2 before the addition
print("Length of dictionary after adding mine: ",len(phage_dict_updated))


Length of dictionary before adding mine:  268
Length of dictionary after adding mine:  2265


In [138]:
#TODO: remove everything that is not a genus in the dictionary

In [137]:
'''with open('../data/interim/updated_gram_pos_neg_dict.pkl', 'wb') as file:
    pickle.dump(phage_dict_updated, file)'''

In [180]:
phage_dict_updated

{'Abyssibacter': 'neg',
 'Abyssicoccus': 'pos',
 'Acanthopleuribacter': 'neg',
 'Acaricomes': 'pos',
 'Acaryochloris': 'neg',
 'Acetanaerobacterium': 'pos',
 'Acetilactobacillus': 'pos',
 'Acetivibrio': 'pos',
 'Acetobacter': 'neg',
 'Acetomicrobium': 'neg',
 'Acholeplasmataceae': 'neg',
 'Achromobacter': 'neg',
 'Acidaminococcus': 'neg',
 'Acidibrevibacterium': 'neg',
 'Acidicapsa': 'neg',
 'Acidihalobacter': 'neg',
 'Acidimangrovimonas': 'neg',
 'Acidimicrobium': 'pos',
 'Acidiphilium': 'neg',
 'Acidipropionibacterium': 'pos',
 'Acidisoma': 'neg',
 'Acidisphaera': 'neg',
 'Acidithiobacillus': 'neg',
 'Acidobacteria': 'neg',
 'Acidobacterium': 'neg',
 'Acidocella': 'neg',
 'Acidovorax': 'neg',
 'Acinetobacter': 'neg',
 'Acrocarpospora': 'pos',
 'Actibacter': 'neg',
 'Actibacterium': 'neg',
 'Actinacidiphila': 'pos',
 'Actinoallomurus': 'pos',
 'Actinoalloteichus': 'pos',
 'Actinobacillus': 'neg',
 'Actinobacteria': 'pos',
 'Actinobaculum': 'pos',
 'Actinocatenispora': 'pos',
 'Actinoc